In [88]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Input, Dense, Flatten, Conv2D, Conv1D,MaxPooling2D, UpSampling2D, MaxPooling1D, BatchNormalization, UpSampling1D
from keras.models import Model

In [98]:
df = pd.read_csv('processedData.csv')
df.head()

/home/pyf/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Elapsed time,Driver,ECG,EMG,foot GSR,hand GSR,HR,marker,RESP,stress,...,foot meanGSR,hand meanGSR,foot meanSCR,hand meanSCR,foot maxSCR,hand maxSCR,foot meanSCL,hand slopeSCL,foot slopeSCL,hand meanSCL
0,'0:00.000',6,-0.002029,0.124,9.051,19.072,90.0,12.36,39.97,1,...,8.941716,18.961706,0.043065,0.074294,0.163574,0.349776,-0.043065,3.293266,3.554833,-0.074294
1,'0:00.002',6,-0.005351,0.124,9.051,19.072,90.0,12.36,39.97,1,...,8.941716,18.961706,0.043065,0.074294,0.163574,0.349776,-0.043065,3.293266,3.554833,-0.074294
2,'0:00.004',6,-0.008661,0.124,9.051,19.072,90.0,12.36,39.97,1,...,8.941716,18.961706,0.043065,0.074294,0.163574,0.349776,-0.043065,3.293266,3.554833,-0.074294
3,'0:00.006',6,-0.011926,0.124,9.051,19.072,90.0,12.36,39.97,1,...,8.941716,18.961706,0.043065,0.074294,0.163574,0.349776,-0.043065,3.293266,3.554833,-0.074294
4,'0:00.008',6,-0.015088,0.124,9.051,19.072,90.0,12.36,39.97,1,...,8.941716,18.961706,0.043065,0.074294,0.163574,0.349776,-0.043065,3.293266,3.554833,-0.074294


In [36]:
def train_test(df, norm = False):
    X = df.drop(['Elapsed time', 'Driver', 'marker', 'stress'], axis = 1)
    y = df['stress']
    if norm:
        X=(X-X.min())/(X.max()-X.min()) # Normalising to rescale all the features. 
    encoder = LabelEncoder()
    encoder.fit(y)
    encoded_Y = encoder.transform(y)
    trainX, testX, trainy, testy = train_test_split(X, encoded_Y, test_size = 0.2 )
    num_features = len(X.columns)
    return trainX, testX, trainy, testy, num_features

def make_model(input_shape):
    '''
    This architecture is the same architecture as proposed by Azar et al.
    '''
    model = Sequential()
    model.add(Dense(60, input_dim=input_shape, activation='relu'))
    model.add(Dense(60, activation = 'relu'))
    model.add(Dense(60, activation = 'relu'))
    model.add(Dense(60, activation = 'relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam' ,metrics=['accuracy'])
    model.summary()
    return model

## Training model without Encoding:

In [27]:
trainX, testX, trainy, testy, num_features = train_test(df)
model = make_model(input_shape = num_features)
model.fit(trainX, trainy, epochs = 125,shuffle=True ,validation_split = 0.2)
op = model.evaluate(testX, testy)
print("Test Accuracy=", op[1]*100, '%')
print("Test Loss=", op[0])
model.save('NN_v1.h5')

## Encoding data using Autoencoders

Training autoencoder:

In [55]:
encoding_dim = 5
input_dim = trainX.shape[1]

input_vec = Input(shape=(input_dim,))
encoded = Dense(32, activation='relu')(input_vec)
encoded = Dense(16, activation='relu')(encoded)
encoded = Dense(8, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)


decoded = Dense(8, activation='sigmoid')(encoded)
decoded = Dense(16, activation='sigmoid')(decoded)
decoded = Dense(32, activation='sigmoid')(decoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)


# encoder = Model(input_vec, encoded)
# encoded_input = Input(shape=(encoding_dim,))
# decoder_layer = autoencoder.layers[5]
# decoder = Model(encoded_input, decoder_layer(encoded_input))
# decoder.summary()

autoencoder = Model(input_vec, decoded)
autoencoder.summary()

autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
autoencoder.fit(trainX, trainX,
                epochs=100,
                batch_size=256,
                shuffle=True)

ValueError: Output tensors to a Model must be the output of a Keras `Layer` (thus holding past layer metadata). Found: <keras.layers.core.Dense object at 0x7fc190554f98>

Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 26)                0         
_________________________________________________________________
dense_54 (Dense)             (None, 32)                864       
_________________________________________________________________
dense_55 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_56 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_57 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_58 (Dense)             (None, 8)                 48        
_________________________________________________________________
dense_59 (Dense)             (None, 16)                144

Encoding and decoding training data:

In [58]:
trainX_anc = autoencoder.predict(trainX) # Encoding and decoding the training data
testX_anc = autoencoder.predict(testX) # Encoding and decoding the testing data
print(trainX_anc.shape)
print(testX_anc.shape)

(38436, 26)
(9609, 26)


**Training NN model on encoded-decoded data**

In [59]:
postEnc_model = make_model(input_shape = trainX_anc.shape[1])
postEnc_model.fit(trainX_anc, trainy, epochs = 125,shuffle=True ,validation_split = 0.2)
op = postEnc_model.evaluate(testX_anc, testy)
print("Test Accuracy after encoding-decoding =", op[1]*100, '%')
print("Test Loss=", op[0])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_123 (Dense)            (None, 60)                1620      
_________________________________________________________________
dense_124 (Dense)            (None, 60)                3660      
_________________________________________________________________
dense_125 (Dense)            (None, 60)                3660      
_________________________________________________________________
dense_126 (Dense)            (None, 60)                3660      
_________________________________________________________________
dense_127 (Dense)            (None, 1)                 61        
Total params: 12,661
Trainable params: 12,661
Non-trainable params: 0
_________________________________________________________________
Train on 30748 samples, validate on 7688 samples
Epoch 1/125
30748/30748 [==============================] - 4s 115us

Epoch 103/125
30748/30748 [==============================] - 3s 104us/step - loss: 0.4898 - accuracy: 0.7539 - val_loss: 0.4860 - val_accuracy: 0.7605
Epoch 104/125
30748/30748 [==============================] - 3s 103us/step - loss: 0.4904 - accuracy: 0.7538 - val_loss: 0.4896 - val_accuracy: 0.7516
Epoch 105/125
30748/30748 [==============================] - 3s 100us/step - loss: 0.4891 - accuracy: 0.7551 - val_loss: 0.4864 - val_accuracy: 0.7653
Epoch 106/125
30748/30748 [==============================] - 3s 102us/step - loss: 0.4905 - accuracy: 0.7546 - val_loss: 0.4913 - val_accuracy: 0.7581
Epoch 107/125
30748/30748 [==============================] - 3s 100us/step - loss: 0.4911 - accuracy: 0.7542 - val_loss: 0.4872 - val_accuracy: 0.7572
Epoch 108/125
30748/30748 [==============================] - 3s 101us/step - loss: 0.4894 - accuracy: 0.7534 - val_loss: 0.5024 - val_accuracy: 0.7530
Epoch 109/125
30748/30748 [==============================] - 3s 100us/step - loss: 0.4899 - ac

**Accuracy went down to 75% (from 95% acc without encoding data) after encoding-decoding the data**

### Autoencoder v2

In [77]:
encoding_dim = 5
input_dim = trainX.shape[1]

input_vec = Input(shape=(input_dim,))
encoded = Dense(16, activation='relu')(input_vec)
encoded = Dense(16, activation='relu')(encoded)
encoded = Dense(16, activation='relu')(encoded)
encoded = Dense(8, activation='relu')(encoded)
encoded = Dense(8, activation='relu')(encoded)
encoded = Dense(8, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)

decoded = Dense(8, activation='sigmoid')(encoded)
decoded = Dense(8, activation='sigmoid')(decoded)
decoded = Dense(8, activation='sigmoid')(decoded)
decoded = Dense(16, activation='sigmoid')(decoded)
decoded = Dense(16, activation='sigmoid')(decoded)
decoded = Dense(16, activation='sigmoid')(decoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

autoencoder2 = Model(input_vec, decoded)
autoencoder2.summary()

autoencoder2.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder2.fit(trainX, trainX,
                epochs=100,
                batch_size=256,
                shuffle=True)

Model: "model_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_35 (InputLayer)        (None, 26)                0         
_________________________________________________________________
dense_208 (Dense)            (None, 16)                432       
_________________________________________________________________
dense_209 (Dense)            (None, 16)                272       
_________________________________________________________________
dense_210 (Dense)            (None, 16)                272       
_________________________________________________________________
dense_211 (Dense)            (None, 8)                 136       
_________________________________________________________________
dense_212 (Dense)            (None, 8)                 72        
_________________________________________________________________
dense_213 (Dense)            (None, 8)                 72 

In [75]:
trainX_anc = autoencoder2.predict(trainX) # Encoding and decoding the training data
testX_anc = autoencoder2.predict(testX) # Encoding and decoding the testing data
print(trainX_anc.shape)
print(testX_anc.shape)

model_enc2 = make_model(input_shape = trainX_anc.shape[1]) 
model_enc2.fit(trainX_anc, trainy, epochs = 100,shuffle=True ,validation_split = 0.2)
op = model_enc2.evaluate(testX_anc, testy)
print("Test Accuracy after encoding-decoding =", op[1]*100, '%')
print("Test Loss=", op[0])

(38436, 26)
(9609, 26)
Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_189 (Dense)            (None, 60)                1620      
_________________________________________________________________
dense_190 (Dense)            (None, 60)                3660      
_________________________________________________________________
dense_191 (Dense)            (None, 60)                3660      
_________________________________________________________________
dense_192 (Dense)            (None, 60)                3660      
_________________________________________________________________
dense_193 (Dense)            (None, 1)                 61        
Total params: 12,661
Trainable params: 12,661
Non-trainable params: 0
_________________________________________________________________
Train on 30748 samples, validate on 7688 samples
Epoch 1/100
30748/30748 [===================

KeyboardInterrupt: 

## Autoencoders v3
Here, we will focus on compressing the depth (i.e. number of records) in the dataset rather than compressing the breath (i.e. dimensions or number of features). 
This is based on this (https://www.sciencedirect.com/science/article/pii/S1389041718302730) approach

In [37]:
dfv3 = df.copy(deep = True)
drivers = list(dfv3['Driver'].value_counts().keys())
dfv3['Driver'].value_counts()

17b    4960
16     4960
11     4960
10     4960
9      4960
8      4960
7      4960
6      4960
17a    4800
12     3008
12      557
Name: Driver, dtype: int64

In [96]:
def CAE(time_steps = 2000):
    
    input_sample = Input(shape=(time_steps, 1))
    encoding = Conv1D(filters=8, kernel_size=3, activation='relu', padding = 'same')(input_sample)
    encoding = MaxPooling1D(pool_size = 2, padding = 'same')(encoding)
    encoding = Conv1D(filters = 32, kernel_size = 5, activation = 'relu', padding = 'same')(encoding)
    encoding = BatchNormalization()(encoding)
    encoding = MaxPooling1D(pool_size = 2, padding = 'same')(encoding)
    
    encoding = Conv1D(filters = 16, kernel_size = 3, activation = 'relu', padding = 'same')(encoding)
    encoding = BatchNormalization()(encoding)
    encoding = MaxPooling1D(pool_size = 2, padding = 'same')(encoding)
  
    encoding = Conv1D(filters = 64, kernel_size = 11, activation = 'relu', padding = 'same')(encoding)
    encoding = Conv1D(filters = 128, kernel_size = 13, activation = 'relu', padding = 'same')(encoding)
    encoding = MaxPooling1D(pool_size = 2, padding = 'same')(encoding)
    encoding = Conv1D(filters = 32, kernel_size = 3, activation = 'relu', padding = 'same')(encoding)
    encoding = Conv1D(filters = 1, kernel_size = 7, activation = 'relu', padding = 'same')(encoding)
    encoding = MaxPooling1D(pool_size = 2, padding = 'same')(encoding)
    
    # Making decoder:
    decoding = Conv1D(filters = 1, kernel_size = 7, padding='same', activation = 'relu')(encoding)
    decoding = Conv1D(32, 3, activation = 'relu', padding = 'same')(decoding)
    decoding = UpSampling1D()(decoding)
    decoding = Conv1D(64, 11, activation = 'relu')(decoding)
    decoding = Conv1D(128, 13, activation = 'relu')(decoding)
    decoding = UpSampling1D()(decoding)
    decoding = Conv1D(16, 3, activation = 'relu')(decoding)
    decoding = Conv1D(32, 5, activation = 'relu')(decoding)
    decoding = UpSampling1D()(decoding)
    decoding = Conv1D(32, 3, activation = 'relu')(decoding)
    decoding = UpSampling1D()(decoding)
    decoding = Conv1D(8, 3, activation = 'relu')(decoding)
    decoding = Flatten()(decoding)
    decoding = Dense(time_steps, activation = 'sigmoid')(decoding)
    
    model = Model(input_sample, decoding)
    model.compile(optimizer='adadelta', loss='mean_squared_error')
    
    # Making a seperate encoder 
    encoder = Model(input_sample, encoding)
    
    return model, encoder

mdl, encoder = CAE()
mdl.summary()

x = np.array(dfv3.loc[:1999, ['ECG']])
x = x.reshape(-1, 2000, 1)
x.shape
mdl.fit(x, x.reshape(-1, 2000))

temp = encoder.predict(x)

Model: "model_44"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_55 (InputLayer)        (None, 2000, 1)           0         
_________________________________________________________________
conv1d_207 (Conv1D)          (None, 2000, 8)           32        
_________________________________________________________________
max_pooling1d_73 (MaxPooling (None, 1000, 8)           0         
_________________________________________________________________
conv1d_208 (Conv1D)          (None, 1000, 32)          1312      
_________________________________________________________________
batch_normalization_32 (Batc (None, 1000, 32)          128       
_________________________________________________________________
max_pooling1d_74 (MaxPooling (None, 500, 32)           0         
_________________________________________________________________
conv1d_209 (Conv1D)          (None, 500, 16)           155

In [97]:
temp.shape

(1, 63, 1)